In [1]:
import pandas as pd
import requests
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("GmapAPIkey")
ADDRESS = "japan"
# ADDRESS = input()


In [2]:
url = f"https://maps.googleapis.com/maps/api/geocode/json?address={ADDRESS}&key={API_KEY}"
query = 'times square'
url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={query}&key={API_KEY}'

res = requests.get(url)
locations = res.json()["results"]
print(locations)
for location in locations:
    print(location)
    location = location["geometry"]["location"]
    lat = location["lat"]
    lng = location["lng"]
    print('-'*30)
    print(f'place latitude:{lat}, longitude:{lng}')

url = f"https://maps.googleapis.com/maps/api/staticmap?center={lat},{lng}&zoom=16&size=400x400&markers=color:red%7C{lat},{lng}&key={API_KEY}"

res = requests.get(url)

with open("skytree.png", "wb") as f:
    f.write(res.content)

[{'business_status': 'OPERATIONAL', 'formatted_address': 'Manhattan, NY 10036, United States', 'geometry': {'location': {'lat': 40.7579747, 'lng': -73.9855426}, 'viewport': {'northeast': {'lat': 40.76098665, 'lng': -73.9834052}, 'southwest': {'lat': 40.75504125, 'lng': -73.98769160000002}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Times Square', 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112698735986262334522">Faisal Baig</a>'], 'photo_reference': 'AUc7tXXWntP2ICIpMr6r1Wt_nGYZQYYLcbPeSFFPECPN8e5S19SxCev5y8ANj3f5Q0iutl0Na59G19PZf2ufm4oBcTyVUxXkUJ5IYjMlFdJWJGuOp4lD4d-BT-Gkzq444lnKGOJtoWdqkd5JDwTIQr8tJPfy00XGAI2fzR1cKsP0g48UrC8Q', 'width': 3000}], 'place_id': 'ChIJmQJIxlVYwokRLgeuocVOGVU', 'plus_code': {'compound_code': 'Q257+5Q Manhattan, New Yo

In [3]:
import requests
from shapely.geometry import Point

def get_times_square_locations(api_key, query='times square'):
    # Google Places APIのエンドポイント
    url = f'https://maps.googleapis.com/maps/api/place/textsearch/json'
    names = []
    lats = []
    lngs = []
    # 検索する場所のリスト（）
    locations_to_search = [
        {'location': '40.7580,-73.9855', 'radius': 10000},  # 北米
        {'location': '-16.063049, -59.941218', 'radius': 5000},  # 南米
        {'location': '24.182634, 55.213924', 'radius': 50000}, # 中東
        {'location': '56.116630, 25.516148', 'radius': 5000},  # ヨーロッパ
        {'location': '0.942591, 18.697926', 'radius': 5000}, # アフリカ
        {'location': '37.5665,126.9780', 'radius': 30000}   # アジア
    ]
    
    locations = []
    for loc in locations_to_search:
        params = {
            'query': query,
            'location': loc['location'],
            'radius': loc['radius'],
            'type': 'tourist_attraction',
            'key': api_key
        }
        
        while True:
            # APIリクエストを送信
            response = requests.get(url, params=params)
            results = response.json().get('results', [])
            # 結果を解析して、緯度と経度を取得
            for result in results:
                print("-"*30)
                print(result)
                names.append(result.get('name'))
                lats.append(result['geometry']['location']['lat'])
                lngs.append(result['geometry']['location']['lng'])
                
#                 locations.append({'name': name, 'lat': lat, 'lng': lng})
            
            # 次のページのトークンを取得
            next_page_token = response.json().get('next_page_token')
            if not next_page_token:
                break
            
            # 次のページをリクエストするためのパラメータを更新
            params['pagetoken'] = next_page_token
    df = pd.DataFrame({'name':names,'latitude':lats,'longitude':lngs})
    return df

# 使用するAPIキーを指定
API_KEY = API_KEY

# 「タイムズスクエア」という名前のつく場所のリストを取得
times_square_locations = get_times_square_locations(API_KEY)

------------------------------
{'business_status': 'OPERATIONAL', 'formatted_address': 'Manhattan, NY 10036', 'geometry': {'location': {'lat': 40.7579747, 'lng': -73.9855426}, 'viewport': {'northeast': {'lat': 40.76098665, 'lng': -73.9834052}, 'southwest': {'lat': 40.75504125, 'lng': -73.98769160000002}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Times Square', 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112698735986262334522">Faisal Baig</a>'], 'photo_reference': 'AUc7tXVoWbF4T7JdB1DNB4NlAFlxRmtu6VBrBMwxvIFSId4J5zO2bYghLB4arKq0u0tGmyxxyg__uFW_zcuc4As3c6YGsNUfQU0aoVfgGPEOZsOEeVX_tR1M94OuOoUzs3ufwVnw6sg8AzehG6tsuJeEjIELTGL4VyUh-HTBMyaOJXwf4HPs', 'width': 3000}], 'place_id': 'ChIJmQJIxlVYwokRLgeuocVOGVU', 'plus_code': {'compound_code': 'Q257+5Q Ma

In [4]:
df = times_square_locations.drop_duplicates().reset_index(drop=True)
df

,name,latitude,longitude
0,Times Square,40.757975,-73.985543
1,Father Duffy Square,40.759008,-73.985022
2,Top of The Rock,40.759349,-73.979409
3,Madame Tussauds New York,40.756427,-73.988834
4,One Times Square,40.756374,-73.986455
5,Town Square,40.758862,-73.984757
6,The Channel Gardens,40.758501,-73.978087
7,Times Square 35 Times,40.755376,-73.987443
8,RiseNY,40.757621,-73.985176
9,Door to Nowhere,40.755639,-73.986354


In [5]:
!pip install pygeocoder
!pip install googlemaps

  Preparing metadata (setup.py) ... - done
  Created wheel for pygeocoder: filename=pygeocoder-1.2.5-py3-none-any.whl size=8891 sha256=291705d0f8e5fc3b55895fc5b719cbdb80b1431e93acff73d69216852a179e0e
  Stored in directory: /root/.cache/pip/wheels/ca/d6/96/20798220a7649a722a405557f09f7dc37a3038a8822fcdda5b
Successfully built pygeocoder
  Preparing metadata (setup.py) ... - \ done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=5c6389025d5f26698788b58555ee83a3c96b3106a35e8aa0704adb0ff4d7fa33
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [6]:
# Google API モジュール
# from pygeocoder import Geocoder
import googlemaps


In [7]:
gmaps = googlemaps.Client(key=API_KEY)
address = 'タイムズスクエア'
results = gmaps.geocode(address)
results

[{'address_components': [{'long_name': 'Manhattan',
    'short_name': 'Manhattan',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'New York',
    'short_name': 'New York',
    'types': ['locality', 'political']},
   {'long_name': 'New York County',
    'short_name': 'New York County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '10036', 'short_name': '10036', 'types': ['postal_code']}],
  'formatted_address': 'Manhattan, NY 10036, USA',
  'geometry': {'location': {'lat': 40.7579747, 'lng': -73.9855426},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': 40.7595003, 'lng': -73.9841994197085},
    'southwest': {'lat': 40.7565276, 'lng': -73.98689738029152}}},
  'partial_match': True,
  

https://qiita.com/Spooky_Maskman/items/9f4c487ed884d803641b
https://qiita.com/yoshi_yast/items/bb75d8fceb712f1f49d1